In [1]:
import pandas as pd
import numpy as np
import nltk
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.tokenize import word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer
from tqdm.notebook import tqdm

In [2]:
df = pd.read_csv('data/tweets_ecs.csv')

In [3]:
temp_detokenize = []

def detokenize(text):
    text1 = text.replace(']','').replace('[','')
    arr = text1.replace('"','').replace("\'","").split(",")
    return(TreebankWordDetokenizer().detokenize(arr))

df['tweet'] = df['tweet'].astype('U').apply(detokenize)

In [4]:
word_dict = {}
for i in range(0,len(df['tweet'])):
    sentence = df['tweet'][i]
    word_token = word_tokenize(sentence)
    for j in word_token:
        if j not in word_dict:
            word_dict[j] = 1
        else:
            word_dict[j] += 1
            
print("Kata unik pada Dataset : "+str(len(word_dict)))

Kata unik pada Dataset : 6013


In [5]:
lexicon = pd.read_csv('data/lexicon/full_lexicon.csv')
lexicon = lexicon.drop(lexicon[(lexicon['word'] == 'bukan')|(lexicon['word'] == 'tidak')|(lexicon['word'] == 'ga')|(lexicon['word'] == 'gk') ].index,axis=0)
lexicon = lexicon.reset_index(drop=True)
len(lexicon)
print("Kamus pada corpus (lexicon) : "+str(len(lexicon)))

Kamus pada corpus (lexicon) : 11930


In [6]:
lexicon_word = lexicon['word'].to_list()
lexicon_num_words = lexicon['number_of_words']
print("Jumlah setiap kata pada corpus :")
lexicon['number_of_words'].value_counts()

Jumlah setiap kata pada corpus :


1    11190
2      684
3       26
4       25
5        5
Name: number_of_words, dtype: int64

In [7]:
ns_words = []
factory = StemmerFactory()
stemmer = factory.create_stemmer()
for word in tqdm(word_dict.keys(),desc='Progress'):
    if word not in lexicon_word:
        kata_dasar = stemmer.stem(word)
        if kata_dasar not in lexicon_word:
            ns_words.append(word)
len(ns_words)

Progress:   0%|          | 0/6013 [00:00<?, ?it/s]

4618

In [8]:
ns_words_list = {k:v for (k,v) in word_dict.items() if ((k in ns_words)&(v>3))}
sort_orders = sorted(ns_words_list.items(), key=lambda x: x[1], reverse=True)
sort_orders=sort_orders[0:20]
for i in sort_orders:
    print(i[0], i[1])

telur 2955
ternak 1019
telor 646
jagung 367
pakan 337
rp 240
kg 229
pasar 198
order 170
pempek 169
lenjer 167
rm 154
dm 145
ribu 141
blitar 141
kilo 107
kulit 103
pakai 103
palembang 100
tan 97


In [9]:
negasi = ['bukan','tidak','ga','gk']
sencol =[]
senrow =np.array([])
nsen = 0
factory = StemmerFactory()
stemmer = factory.create_stemmer()
sentiment_list = []
# function to write the word's sentiment if it is founded
def found_word(ind,words,word,sen,sencol,sentiment,add):
    # if it is already included in the bag of words matrix, then just increase the value
    if word in sencol:
        sen[sencol.index(word)] += 1
    else:
    #if not, than add new word
        sencol.append(word)
        sen.append(1)
        add += 1
    #if there is a negation word before it, the sentiment would be the negation of it's sentiment
    if (words[ind-1] in negasi):
        sentiment += -lexicon['weight'][lexicon_word.index(word)]
    else:
        sentiment += lexicon['weight'][lexicon_word.index(word)]
    
    return sen,sencol,sentiment,add
            
# checking every words, if they are appear in the lexicon, and then calculate their sentiment if they do
for i in tqdm(range(len(df)),desc='Progress'):
    nsen = senrow.shape[0]
    words = word_tokenize(df['tweet'][i])
    sentiment = 0 
    add = 0
    prev = [0 for ii in range(len(words))]
    n_words = len(words)
    if len(sencol)>0:
        sen =[0 for j in range(len(sencol))]
    else:
        sen =[]
    
    for word in words:
        ind = words.index(word)
        # check whether they are included in the lexicon
        if word in lexicon_word :
            sen,sencol,sentiment,add= found_word(ind,words,word,sen,sencol,sentiment,add)
        else:
        # if not, then check the root word
            kata_dasar = stemmer.stem(word)
            if kata_dasar in lexicon_word:
                sen,sencol,sentiment,add= found_word(ind,words,kata_dasar,sen,sencol,sentiment,add)
        # if still negative, try to match the combination of words with the adjacent words
            elif(n_words>1):
                if ind-1>-1:
                    back_1    = words[ind-1]+' '+word
                    if (back_1 in lexicon_word):
                        sen,sencol,sentiment,add= found_word(ind,words,back_1,sen,sencol,sentiment,add)
                    elif(ind-2>-1):
                        back_2    = words[ind-2]+' '+back_1
                        if back_2 in lexicon_word:
                            sen,sencol,sentiment,add= found_word(ind,words,back_2,sen,sencol,sentiment,add)
    # if there is new word founded, then expand the matrix
    if add>0:  
        if i>0:
            if (nsen==0):
                senrow = np.zeros([i,add],dtype=int)
            elif(i!=nsen):
                padding_h = np.zeros([nsen,add],dtype=int)
                senrow = np.hstack((senrow,padding_h))
                padding_v = np.zeros([(i-nsen),senrow.shape[1]],dtype=int)
                senrow = np.vstack((senrow,padding_v))
            else:
                padding =np.zeros([nsen,add],dtype=int)
                senrow = np.hstack((senrow,padding))
            senrow = np.vstack((senrow,sen))
        if i==0:
            senrow = np.array(sen).reshape(1,len(sen))
    # if there isn't then just update the old matrix
    elif(nsen>0):
        senrow = np.vstack((senrow,sen))
        
    sentiment_list.append(sentiment)
    
len(sentiment_list)

Progress:   0%|          | 0/2879 [00:00<?, ?it/s]

2879

In [10]:
sencol.append('sentiment')
sentiment_array = np.array(sentiment_list).reshape(senrow.shape[0],1)
sentiment_data = np.hstack((senrow,sentiment_array))

df_sen = pd.DataFrame(sentiment_data,columns = sencol)
df_sen

,ayam,diri,atas,sendiri,stabil,harga,sebab,anjlok,kaget,turun,...,mood,konyol,riil,mampir,lagak,hero,tembak,sendal,sembahyang,sentiment
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-2
1,0,1,1,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-2
2,0,0,0,0,0,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,5
3,3,0,0,0,0,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,-1
4,3,0,0,0,0,2,1,0,0,1,...,0,0,0,0,0,0,0,0,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2874,1,0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,-7
2875,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,9
2876,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,23
2877,0,0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,-5


In [11]:
# melihat sentimen dari tweet original

cek_df = pd.DataFrame()
temp_cek_df = []

cek_df['tweet'] = df['tweet'].copy()
cek_df['polaritas']  = df_sen['sentiment'].copy()

for senti in df_sen['sentiment']:
    if (int(senti) >= 0):
        label = "Positif"
    # elif (int(senti) == 0):
    #     label = "Netral"    
    else:
        label = "Negatif"
    temp_cek_df.append([label])
    
temp_cek_df = pd.DataFrame(temp_cek_df, columns=['sentimen'])
cek_df['sentimen'] = temp_cek_df['sentimen'].copy()
cek_df.reset_index(drop=True, inplace=True)
cek_df

,tweet,polaritas,sentimen
0,jbharga ayam rm tgk telor wyam papan rm...,-2,Negatif
1,diri di atas kaki sendiri serap telur t...,-2,Negatif
2,sebab harga telur anjlok versi dagang kaget,5,Positif
3,turun minta masyarakat telur ayam ppkm l...,-1,Negatif
4,ternak ayam telur menga rugi ratus juta ...,4,Positif
...,...,...,...
2874,blitar harga telur ayam anjlok ternak an...,-7,Negatif
2875,titip sendal baca artikel nasi goreng go...,9,Positif
2876,suka mooncake isi telur daerah mahal mah...,23,Positif
2877,harga telur anjlok ternak jateng babak b...,-5,Negatif


In [12]:
print("Positif :",len(cek_df[cek_df.polaritas>=0]), " tweet")
print("Negatif :",len(cek_df[cek_df.polaritas<0]), " tweet")

Positif : 2207  tweet
Negatif : 672  tweet


In [13]:
cek_df.to_csv("data/tweets_labelled_ecs.csv", index=False)

In [14]:
nltk.download('punkt')
def word_tokenize_wrapper(text):
    return word_tokenize(text)

cek_df['tweet'] = cek_df['tweet'].apply(word_tokenize_wrapper)
cek_df.to_csv("data/tweets_labelled_tokenized_ecs.csv", index=False)

[nltk_data] Downloading package punkt to /home/anz007/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
